In [ ]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py



In [ ]:
plt.rcParams["figure.figsize"] = (16,12)
plt.rcParams["font.size"] = 24
plt.rcParams['xtick.labelsize'] = "small"
plt.rcParams['ytick.labelsize'] = 36

In [ ]:
def load_dir_reduced(dirname,file_prefix,max_files):
    '''
    Load time information from the h5 files in a loop into a list. Step size is fixed to 100. 
    '''   
    ## Load all filenames in directory
    var_list1 = []
    var_list2 = []
    var_list3 = []
    files = []
    [files.append(file_) for file_ in os.listdir(dirname) if file_.startswith(file_prefix) if file_.endswith('.h5')]
    files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
    print(len(files))
    step_size = 50
    for j in np.arange(0,max_files,step_size):
        for filename in files[j:j+step_size]:
            BDFs = BDF.BeadDataFile(dirname+filename)
            var_list1.append(BDFs.x2)
            var_list2.append(BDFs.cant_pos[1])
            var_list3.append(BDFs.y2)
    return var_list1,var_list2,var_list3

In [ ]:
def data_to_amp_and_phase_single_axis(data,fsamp,res):
    data_det=signal.detrend(data)
    freqs=np.linspace(0,int(fsamp/2),(int(res/2))+1)    # change
    FFT=np.fft.rfft(data_det)
    norm = np.sqrt(2 / (res* fsamp))
    PSD = norm**2 * (FFT * FFT.conj()).real
    Phase =np.angle(FFT)
    return PSD,Phase

## verification of the methods comparability to previously used mlab.psd

# fsamp=5000
# res=50000
# data=f[0].x2
# a = data_to_amp_and_phase_single_axis(data,fsamp,res)
# b = matplotlib.mlab.psd(signal.detrend(data), Fs = fsamp, NFFT = res, window = mlab.window_none)
# plt.loglog(a[0],a[1])
# plt.loglog(b[1],b[0],alpha=0.5)

In [ ]:
def loop_extract_PSD_and_phase(inList,fsamp,res,calibrationFactor=1):
    ampList = []
    phaseList=[]
    for i in range(len(inList)):
        data=inList[i]/calibrationFactor
        temp_ = data_to_amp_and_phase_single_axis(data,fsamp,res)
        ampList.append(temp_[0])
        phaseList.append(temp_[1])
    return ampList,phaseList

In [ ]:
def extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=1,calibrationFactorY=1):
    start=time.time()

    df= pd.DataFrame()
    fsamp=5000
    res=50000
    freqs=np.linspace(0,int(fsamp/2),(int(res/2))+1)
    x2L,cPL,y2L = load_dir_reduced(folderName,file_prefix=filePrefix,max_files=maxFiles)
    xAmpList,xPhaseList = loop_extract_PSD_and_phase(x2L,fsamp,res,calibrationFactorX)
    yAmpList,yPhaseList = loop_extract_PSD_and_phase(y2L,fsamp,res,calibrationFactorY)
    
    df["cantPosY"] = cPL
    
    df["xAmp"] = x2L
    df["xPhase"] = xPhaseList
    df["xPSD"] = xAmpList
    #df["xASD"]=df["xPSD"].apply(lambda element: np.sqrt(element))
    
    df["yAmp"] = y2L
    df["yPhase"] = yPhaseList
    df["yPSD"] = yAmpList
    #df["yASD"]=df["yPSD"].apply(lambda element: np.sqrt(element))
    
    
    df["checkStd"]=df["xAmp"].apply(lambda element: np.std(element))
    

    if(filterStd==True):
        df=df[df["checkStd"]<20*df["checkStd"].median()]
    print("The process took %.2f" %(time.time()-start))
    print(df.info(memory_usage='deep))
    return freqs,df

In [ ]:
def plotQuickPSD(fundamental,df_1,df_2,axis,label1,label2,offset,lowylim=1e-10,upylim=1e-4,lowxlim=10,upxlim=150,scaleFactor1=1,scaleFactor2=1,res=5000,fsamp=5000):

    xlabel="frequency [Hz]"
    ylabel="%s sumPSD [a.u.]" %axis
    
    plt.plot(freqs,df_1["%sPSD" %axis].sum()/scaleFactor1,color="black",alpha=1,lw=3,label=label1)
    plt.plot(freqs,df_2["%sPSD" %axis].sum()/scaleFactor2,color="red",alpha=0.7,lw=3,label=label2)
    plt.yscale("log")
    plt.xlim(lowxlim,upxlim)
    plt.ylim(lowylim,upylim)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    plt.legend()
    plt.show()
    
    plt.plot(freqs[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],(df_1["%sPSD" %axis].sum()/scaleFactor1)[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],color="black",alpha=1,lw=3,marker="*",ms=15,label=label1)
    plt.plot(freqs[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],(df_2["%sPSD" %axis].sum()/scaleFactor2)[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],color="red",alpha=0.7,lw=3,marker="*",ms=15,label=label2)
    
    plt.plot(freqs[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],(df_1["%sPSD" %axis].sum()/scaleFactor1)[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],color="blue",alpha=0.7,lw=3,ls="dashed",marker="*",ms=15,label=label1+" off-axis check at %d" %(fundamental+offset))
    plt.plot(freqs[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],(df_2["%sPSD" %axis].sum()/scaleFactor2)[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],color="blue",alpha=0.7,lw=3,ls="dotted",marker="*",ms=15,label=label2+" off-axis check at %d"%(fundamental+offset))

    plt.yscale("log")
    plt.xlim(lowxlim,upxlim)
    plt.ylim(lowylim,upylim)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

In [ ]:
def plotQuickASD(fundamental,df_1,df_2,axis,label1,label2,offset,lowylim=1e-10,upylim=1e-4,lowxlim=10,upxlim=150,scaleFactor1=1,scaleFactor2=1,res=50000,fsamp=5000):

    xlabel="frequency [Hz]"
    ylabel="%s normalized ASD [m/$\sqrt{Hz}$]" %axis
    
        
    plt.plot(freqs,np.sqrt(df_1["%sPSD" %axis].sum()/scaleFactor1),color="black",alpha=1,lw=3,label=label1)
    plt.plot(freqs,np.sqrt(df_2["%sPSD" %axis].sum()/scaleFactor2),color="red",alpha=0.7,lw=3,label=label2)
    plt.yscale("log")
    plt.xlim(lowxlim,upxlim)
    plt.ylim(lowylim,upylim)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    plt.legend()
    plt.show()
    
    plt.plot(freqs[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],(np.sqrt(df_1["%sPSD" %axis].sum()/scaleFactor1))[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],color="black",alpha=1,lw=3,marker="*",ms=15,label=label1)
    plt.plot(freqs[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],(np.sqrt(df_2["%sPSD" %axis].sum()/scaleFactor2))[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],color="red",alpha=0.7,lw=3,marker="*",ms=15,label=label2)
    
    plt.plot(freqs[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],(np.sqrt(df_1["%sPSD" %axis].sum()/scaleFactor1))[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],color="blue",alpha=0.7,lw=3,ls="dashed",marker="*",ms=15,label=label1+" off-axis check at %d" %(fundamental+offset))
    plt.plot(freqs[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],(np.sqrt(df_2["%sPSD" %axis].sum()/scaleFactor2))[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],color="blue",alpha=0.7,lw=3,ls="dotted",marker="*",ms=15,label=label2+" off-axis check at %d"%(fundamental+offset))

    plt.yscale("log")
    plt.xlim(lowxlim,upxlim)
    plt.ylim(lowylim,upylim)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

In [ ]:
def plotQuickAlternatemethod(fundamental,df_1,df_2,label1,label2,offset,lowylim=1e-10,upylim=1e-4,lowxlim=10,upxlim=150,scaleFactor1=1,scaleFactor2=1,res=50000,fsamp=5000):

    data1=data_to_amp_and_phase_single_axis(df_1["xAmp"].sum(),5000,50000)[0]
    data2=data_to_amp_and_phase_single_axis(df_2["xAmp"].sum(),5000,50000)[0]            
    xlabel="frequency [Hz]"
    ylabel="%s sumPSD [a.u.]"
    plt.plot(freqs,data1/scaleFactor1,color="black",alpha=1,lw=3,label=label1)
    plt.plot(freqs,data2/scaleFactor2,color="red",alpha=0.5,lw=3,label=label2)
    plt.yscale("log")
    plt.xlim(lowxlim,upxlim)
    plt.ylim(lowylim,upylim)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plt.legend()
    plt.show()

    plt.plot(freqs[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],(data1/scaleFactor1)[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],color="black",alpha=1,lw=3,marker="*",ms=15,label=label1)
    plt.plot(freqs[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],(data2/scaleFactor2)[fundamental*int(res/fsamp)::fundamental*int(res/fsamp)],color="red",alpha=0.7,lw=3,marker="*",ms=15,label=label2)

    plt.plot(freqs[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],(data1/scaleFactor1)[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],color="blue",alpha=0.7,lw=3,ls="dashed",marker="*",ms=15,label=label1+" off-axis check at %d" %(fundamental+offset))
    plt.plot(freqs[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],(data2/scaleFactor2)[(fundamental+offset)*int(res/fsamp)::(fundamental+offset)*int(res/fsamp)],color="blue",alpha=0.7,lw=3,ls="dotted",marker="*",ms=15,label=label2+" off-axis check at %d"%(fundamental+offset))

    plt.yscale("log")
    plt.xlim(lowxlim,upxlim)
    plt.ylim(lowylim,upylim)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()                       

In [ ]:
df_NoShaking_20220526 = []
for i in tqdm(range(3)):
    folderName="/data/new_trap/20220526/Bead0/NoShaking/%d/" %(i+1)
    filePrefix="NoShaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_NoShaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    df_NoShaking_20220526.append(df_NoShaking_temp)
    

In [ ]:
df_Shaking_20220526 = []
for i in tqdm(range(7)):
    folderName="/data/new_trap/20220526/Bead0/Shaking/13Hz/NoServo/%d/" %(i+1)
    filePrefix="Shaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_Shaking13Hz_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    print(len(df_Shaking13Hz_temp))   
    df_Shaking_20220526.append(df_Shaking13Hz_temp)


In [ ]:
for i in range(7):
    j=2
    data1=df_NoShaking_20220526[j]
    data2=df_Shaking_20220526[i]
    plotQuickAlternatemethod(13,df_1=data1,df_2=data2,label1="noShaking",label2="Shaking 13 Hz w/o servo, 1.25V n=%d" %(len(data2)),offset=3,lowylim=1e-11,upylim=1e-3,scaleFactor1=len(data1),scaleFactor2=len(data2))

In [ ]:
for i in range(7):
    j=2
    data1=df_NoShaking_20220526[j]
    data2=df_Shaking_20220526[i]
    plotQuickPSD(13,df_1=data1,df_2=data2,axis="x",label1="noShaking",label2="Shaking 13 Hz w/o servo, 1.25V n=%d" %(len(data2)),offset=3,lowylim=1e-21,upylim=1e-15,scaleFactor1=len(data1),scaleFactor2=len(data2))

In [ ]:
for i in range(7):
    j=2
    data1=df_NoShaking_20220526[j]
    data2=df_Shaking_20220526[i]
    plotQuickASD(13,df_1=data1,df_2=data2,axis="x",label1="noShaking",label2="Shaking 13 Hz w/o servo, 1.25V n=%d" %(len(data2)),offset=3,lowylim=1e-10,upylim=1e-7,scaleFactor1=len(data1),scaleFactor2=len(data2))

In [ ]:
data1=df_Shaking_20220526[1]
data2=df_Shaking_20220526[6]
plotQuickAlternatemethod(13,df_1=data1,df_2=data2,label1="before",label2="after",offset=3,lowylim=1e-11,upylim=1e-3,scaleFactor1=len(data1),scaleFactor2=len(data2))

In [ ]:
data1=df_Shaking_20220526[1]
data2=df_Shaking_20220526[6]
plotQuickASD(13,df_1=data1,df_2=data2,axis="y",label1="noShaking",label2="Shaking 13 Hz w/o servo, 1.25V n=%d" %(len(data2)),offset=3,lowylim=1e-10,upylim=1e-7,scaleFactor1=len(data1),scaleFactor2=len(data2))

In [ ]:
df_NoShaking_20220601 = []

for i in range(3):
    folderName="/data/new_trap/20220601/VibrationTest/NoShaking/%d/" %(i)
    filePrefix="NoShaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_NoShaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True)
    print(len(df_NoShaking_temp))   
    df_NoShaking_20220601.append(df_NoShaking_temp)


In [ ]:
df_Shaking_20220601 = []
for i in range(3):
    folderName="/data/new_trap/20220601/VibrationTest/Shaking/%d/" %(i+1)
    filePrefix="Shaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_Shaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True)
    print(len(df_Shaking_temp))   
    df_Shaking_20220601.append(df_Shaking_temp)


In [ ]:
df_NoShaking_20220602 = []
for i in range(1):
    folderName="/data/new_trap/20220602/VibrationTest/NoShaking/%d/" %(i)
    filePrefix="NoShaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_NoShaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True)
    print(len(df_NoShaking_temp))   
    df_NoShaking_20220602.append(df_NoShaking_temp)


In [ ]:
df_Shaking_20220602 = []
for i in range(1):
    folderName="/data/new_trap/20220602/VibrationTest/Shaking/%d/" %(i+1)
    filePrefix="Shaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_Shaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True)
    print(len(df_Shaking_temp))   
    df_Shaking_20220602.append(df_Shaking_temp)


In [ ]:
df_NoShaking_20220603 = []

for i in range(1):
    folderName="/data/new_trap/20220603/VibrationTest/NoShaking/%d/" %(i)
    filePrefix="NoShaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_NoShaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True)
    print(len(df_NoShaking_temp))   
    df_NoShaking_20220603.append(df_NoShaking_temp)


In [ ]:
df_Shaking_20220603 = []
for i in range(1):
    folderName="/data/new_trap/20220603/VibrationTest/Shaking/%d/" %(i+1)
    filePrefix="Shaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_Shaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True)
    print(len(df_Shaking_temp))   
    df_Shaking_20220603.append(df_Shaking_temp)


In [ ]:
df_Shaking_20220605 = []
for i in range(1):
    folderName="/data/new_trap/20220605/VibrationTest/Shaking/%d/" %(i+1)
    filePrefix="Shaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_Shaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    print(len(df_Shaking_temp))   
    df_Shaking_20220605.append(df_Shaking_temp)


In [ ]:
df_NoShaking_20220605 = []

for i in range(1):
    folderName="/data/new_trap/20220605/VibrationTest/NoShaking/%d/" %(i)
    filePrefix="NoShaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_NoShaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    print(len(df_NoShaking_temp))   
    df_NoShaking_20220605.append(df_NoShaking_temp)


In [ ]:
df_NoShaking_20220606 = []
for i in range(2):
    folderName="/data/new_trap/20220606/VibrationTest/NoShaking/%d/" %(i)
    filePrefix="NoShaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_NoShaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    print(len(df_NoShaking_temp))   
    df_NoShaking_20220606.append(df_NoShaking_temp)


In [ ]:
df_Shaking_20220606 = []
for i in range(2):
    folderName="/data/new_trap/20220606/VibrationTest/Shaking/%d/" %(i+1)
    filePrefix="Shaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_Shaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    print(len(df_Shaking_temp))   
    df_Shaking_20220606.append(df_Shaking_temp)


In [ ]:
df_NoShaking_20220609 = []
for i in range(1):
    folderName="/data/new_trap/20220609/VibrationTest/NoShaking/%d/" %(i)
    filePrefix="NoShaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_NoShaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    print(len(df_NoShaking_temp))   
    df_NoShaking_20220609.append(df_NoShaking_temp)
    df

In [ ]:
df_Shaking_20220609 = []
for i in range(3):
    folderName="/data/new_trap/20220609/VibrationTest/Shaking/%d/" %(i+1)
    filePrefix="Shaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_Shaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=5000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    print(len(df_Shaking_temp))   
    df_Shaking_20220609.append(df_Shaking_temp)


In [ ]:
df_Shaking_20220609[2].info(memory_usage="deep")

In [ ]:
data1=df_NoShaking_20220609[0]
data2=df_Shaking_20220609[2]
plotQuickASD(3,df_1=data1,df_2=data2,axis="y",label1="noShaking",label2="Shaking 3 Hz w servo, 4V n=%d" %(len(data2)),offset=2,lowylim=1e-10,upylim=1e-8,lowxlim=1,upxlim=40,scaleFactor1=len(data1),scaleFactor2=len(data2))

In [ ]:
data1=df_NoShaking_20220609[0]
data2=df_Shaking_20220609[2]
plotQuickAlternatemethod(3,df_1=data1,df_2=data2,label1="noShaking",label2="Shaking 3 Hz w servo, 4V n=%d" %(len(data2)),offset=2,lowylim=1e-10,upylim=1e-4,lowxlim=1,upxlim=40,scaleFactor1=len(data1),scaleFactor2=len(data2))

In [ ]:
df_NoShaking_20220613 = []
for i in range(1):
    folderName="/data/new_trap/20220613/NoShaking/%d/" %(i)
    filePrefix="NoShaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_NoShaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    print(len(df_NoShaking_temp))   
    df_NoShaking_20220613.append(df_NoShaking_temp)


In [ ]:
df_Shaking_20220613 = []
for i in range(2):
    folderName="/data/new_trap/20220613/Shaking/%d/" %(i+1)
    filePrefix="Shaking"
    #files = load_dir_sorted(folderName,file_prefix=filePrefix,start_file=0,max_file=1000)
    freqs,df_Shaking_temp= extract_data_to_df(folderName,filePrefix,maxFiles=1000,filterStd=True,calibrationFactorX=6e4,calibrationFactorY=3.5e5)
    print(len(df_Shaking_temp))   
    df_Shaking_20220613.append(df_Shaking_temp)


In [ ]:
data1=df_NoShaking_20220613[0]
data2=df_Shaking_20220613[1]
plotQuickASD(3,df_1=data1,df_2=data2,axis="y",label1="noShaking",label2="Shaking 3 Hz w servo, 4V n=%d" %(len(data2)),offset=2,lowylim=1e-10,upylim=1e-7,lowxlim=1,upxlim=40,scaleFactor1=len(data1),scaleFactor2=len(data2))

In [ ]:
data1=df_NoShaking_20220613[0]
data2=df_Shaking_20220613[1]
plotQuickAlternatemethod(3,df_1=data1,df_2=data2,label1="noShaking",label2="Shaking 3 Hz w servo, 4V n=%d" %(len(data2)),offset=2,lowylim=1e-10,upylim=1e-4,lowxlim=1,upxlim=40,scaleFactor1=len(data1),scaleFactor2=len(data2))